# JSON Functions Testing

## Tasks to do:
* Load JSON file
* Dynamically update specified values within JSON file
* Export the updated JSON as new JSON file

In [178]:
lubuff_output = {2016: {'EMPTOT': 4687.0, 'DU_TOT': 3097.0, 'GISAc': 737.9634277199999}, 2040: {'EMPTOT': 5207.99, 'DU_TOT': 3630.36043698, 'GISAc': 737.9634277199999}}
lu_divers_output = {2016: {'mix_index': 0.49111228436332865}, 2040: {'mix_index': 0.5444755587594678}}
lu_div_agg_output = {2016: {'CommunityType': 0.4246659255260256, 'REGION': 0.9999891200962348},
 2040: {'CommunityType': 0.4687834168845922, 'REGION': 0.9308534957530498}}


acc_project_data = {'WALKDESTSalljob': 5652.140425605537, 'BIKEDESTSalljob': 118198.27607266436, 
                    'AUTODESTSalljob': 713138.993297578, 'TRANDESTSalljob': 88363.14543598615, 
                    'WALKDESTSedu': 4.37580276816609, 'BIKEDESTSedu': 40.37682006920416, 
                    'AUTODESTSedu': 728.3332975778545, 'TRANDESTSedu': 24.736899653979243, 
                    'WALKDESTSpoi2': 22.529602076124565, 'BIKEDESTSpoi2': 190.794785467128, 
                    'AUTODESTSpoi2': 943.9961937716263, 'TRANDESTSpoi2': 122.72797923875432}

acc_agg_data = [{'WALKDESTSpoi2': {2016: {'CommunityType': 23.408610137958803,
    'REGION': 17.147198276435414}}},
 {'BIKEDESTSpoi2': {2016: {'CommunityType': 199.25458633185085,
    'REGION': 140.100123535153}}},
 {'AUTODESTSpoi2': {2016: {'CommunityType': 686.1842636258035,
    'REGION': 520.3276979954376}}},
 {'TRANDESTSpoi2': {2016: {'CommunityType': 122.29326086755307,
    'REGION': 77.93892290099564}}}]

In [190]:
acc_agg_data[0]['WALKDESTSpoi2']

{2016: {'CommunityType': 23.408610137958803, 'REGION': 17.147198276435414}}

In [196]:
import json

in_json = r"C:\Users\dconly\GitRepos\PPA3\testing\json_output\SACOG_ReduceVMT_template.json"

# load json file into python dict
with open(in_json, "r") as j_in:
    pyj = json.load(j_in)
    
# print(json.dumps(pyj, indent=4))  # pretty print/format of json

# convert json file into class for each report for more flexibility in coding/parsing
class VMTReport:
    def __init__(self, in_json, agg_data_csv=None):
        # load json template
        self.loaded_json = self.load_json_file(in_json)
        
        # load agg (region and comm type) data table
        # NOTE - will need other function to process appropriately and get ones specific to each metric
        if agg_data_csv:
            self.df_aggdata = pd.read_csv(agg_data_csv)

        # attributes of all charts
        self.k_charts = 'charts'
        self.chart_features = 'features'
        self.chart_attrs = 'attributes'
        self.type = 'type'

        # regional and community type lookup table geography names
        self.aggdata_ctype = 'CommunityType'
        self.aggdata_regn = 'REGION'
        
        # JSON names for each geography
        self.geo_proj = 'Project'
        self.geo_ctyp = 'Community Type'
        self.geo_regn = 'Region'
        
        self.geogs_list = [self.geo_proj, self.geo_ctyp, self.geo_regn]
        


        # accessibility chart
    
    def load_json_file(self, in_json_file):
        with open(in_json_file, "r") as j_in:
            return json.load(j_in)
        
    def emp_du_chart(self, data=None):
        # chart label keys
        self.chart_jdu = 'Jobs and Dwelling'
        self.label_jobs = 'jobs'
        self.label_du = 'dwellingUnits'
        self.label_year = 'year'
        
        # keys in dict of output data
        self.d_year = 'year'
        self.d_emp = 'EMPTOT'
        self.d_du = 'DU_TOT'
            
        data_keys_list = list(data.keys())
        for i, year in enumerate(data_keys_list):
            # update json values
            
            update_dict = {self.label_year:str(year), 
                           self.label_jobs:data[year][self.d_emp],
                           self.label_du:data[year][self.d_du]
                          }
                
            for lname, val in update_dict.items():
                self.loaded_json[self.k_charts][self.chart_jdu][self.chart_features][i] \
                    [self.chart_attrs][lname] = val
                
    def lu_diversity_chart(self, project_data, aggd_data):
        ###NEEDS TO HAVE REGION AND COMMUNITY TYPE STUFF PUT IN!###
        ### MAYBE CAN HAVE SEPARATE FUNCTION THAT JUST INSERTS AGG AVGS???
        # chart label keys
        self.chart_land_div = 'Land Use Diversity'
        self.label_div_prefix = 'diversity '
        
        # project data keys
        self.d_mixidx = 'mix_index'
        year_data_keys = list(project_data.keys())
        
        # comm type and region data keys
        # aggd_data_keylist = list(aggd_data.keys())
        
        for i, geo_type in enumerate(self.geogs_list):
            update_dict = {self.type: geo_type}
            
            for k, year in enumerate(year_data_keys):
                geo_typdict = {self.geo_proj:project_data[year][self.d_mixidx], 
                     self.geo_ctyp:aggd_data[year][self.aggdata_ctype], 
                     self.geo_regn:aggd_data[year][self.aggdata_regn]}
                
                update_dict[f"{self.label_div_prefix}{year}"] = geo_typdict[geo_type]
            
            # import pdb; pdb.set_trace()
            self.loaded_json[self.k_charts][self.chart_land_div][self.chart_features][i] \
            [self.chart_attrs] = update_dict
            
    def accessibility_chart(self, project_data, aggd_data):
        # each "feature" is an accessibility type
        # each type also has project, ctype, region labels.
        
        self.chart_accsvcs = 'Base Year Service Accessibility'
        
        #{name from data set: name in VGIS json file}
        self.names_dict = {'WALKDESTSpoi2':'30 Min Walk', 
                                'BIKEDESTSpoi2':'30 Min Biking', 
                                'AUTODESTSpoi2':'15 Min Drive', 
                                'TRANDESTSpoi2':'45 Min Transit'}
        
        acc_data_keys = list(self.names_dict.keys())
        
        agg_data_scenarios = [2016] # not really used right now, but building in case we 
        # eventually do before-after accessibility comparison
        
        for i, acctype in enumerate(acc_data_keys):
            update_dict = {self.type:self.names_dict[acctype]} # type
            agg_acc_dict = aggd_data[i][acctype] # dict with acc type, year, commtype value, reg value
            # import pdb; pdb.set_trace()
            
            for scen in agg_data_scenarios:            
                update_dict[self.geo_proj] = project_data[acctype] # project's acc value
                update_dict[self.geo_ctyp] = agg_acc_dict[scen][self.aggdata_ctype]
                update_dict[self.geo_regn] = agg_acc_dict[scen][self.aggdata_regn]
            
            self.loaded_json[self.k_charts][self.chart_accsvcs][self.chart_features][i] \
            [self.chart_attrs] = update_dict
            
                
            
        
    
to = VMTReport(in_json)
to.emp_du_chart(lubuff_output)
to.lu_diversity_chart(project_data=lu_divers_output, aggd_data=lu_div_agg_output)
to.accessibility_chart(project_data=acc_project_data, aggd_data=acc_agg_data)
    

In [197]:
to.loaded_json

{'charts': {'Jobs and Dwelling': {'title': 'Total Jobs and Dwelling Units within 0.5mi of project',
   'features': [{'attributes': {'year': '2016',
      'jobs': 4687.0,
      'dwellingUnits': 3097.0}},
    {'attributes': {'year': '2040',
      'jobs': 5207.99,
      'dwellingUnits': 3630.36043698}}]},
  'Land Use Diversity': {'title': 'Land Use Diversity within 1mi of Project',
   'features': [{'attributes': {'type': 'Project',
      'diversity 2016': 0.49111228436332865,
      'diversity 2040': 0.5444755587594678}},
    {'attributes': {'type': 'Community Type',
      'diversity 2016': 0.4246659255260256,
      'diversity 2040': 0.4687834168845922}},
    {'attributes': {'type': 'Region',
      'diversity 2016': 0.9999891200962348,
      'diversity 2040': 0.9308534957530498}}]},
  'Base Year Service Accessibility': {'title': 'Land Use Diversity within 1mi of Project',
   'features': [{'attributes': {'type': '30 Min Walk',
      'Project': 22.529602076124565,
      'Community Type': 23.

In [110]:
d = {'a':1}
d.update

In [18]:
# output as new JSON file

out_json = json.dumps(pyj, indent=4) # indent spec needed so it doesn't come out as single line and is more readable

output_json = "test_json_out.json"

with open(output_json, 'w') as j_out:
    j_out.write(out_json)

# Pulling out specific values for community and region averages from CSV

In [176]:
import pandas as pd

aggvals_csv = r"C:\Users\dconly\GitRepos\PPA2\ppa\Input_Template\CSV\Agg_ppa_vals04222020_1017.csv"

metric = 'mix_index'  # 'WALKDESTSalljob' # 'mix_index'
project_type = "Arterials & Suburban Corridors"

def get_aggvals(aggvals_csv, metric_name, project_ctype):
    col_metricname = 'metric_name'
    df_aggvals = pd.read_csv(aggvals_csv).rename(columns={'Unnamed: 0': col_metricname})
    col_aggvals_year = 'year'
    region_headname = 'REGION'
    cols_ctype_reg = [project_ctype, region_headname, col_aggvals_year]
    aggval_headers = {col: 'CommunityType' for col in df_aggvals.columns if col != region_headname}

    df2 = df_aggvals.loc[df_aggvals[col_metricname] == metric_name]  # filter to indicated metric
    df2 = df2[cols_ctype_reg]  # only include community types for community types that project is in
    df2 = df2.rename(columns={project_ctype: 'CommunityType'})
    df2 = df2.rename(columns={col:f'{col}' for col in list(df_agg_yr.columns)})

    df2 = df2.set_index(col_aggvals_year)
    dict_out = df2.to_dict(orient='index')
    return {metric_name: dict_out}
    # return dict_out

# td = get_aggvals(aggvals_csv, metric, project_type)
acc_data = [get_aggvals(aggvals_csv, metname, project_type) for metname in \
            ['WALKDESTSpoi2', 'BIKEDESTSpoi2', 'AUTODESTSpoi2', 'TRANDESTSpoi2']]
for d in acc_data:
    k = list(d.keys())[0]
    print(k, d[k][2016])

WALKDESTSpoi2 {'CommunityType': 23.408610137958803, 'REGION': 17.147198276435414}
BIKEDESTSpoi2 {'CommunityType': 199.25458633185085, 'REGION': 140.100123535153}
AUTODESTSpoi2 {'CommunityType': 686.1842636258035, 'REGION': 520.3276979954376}
TRANDESTSpoi2 {'CommunityType': 122.29326086755307, 'REGION': 77.93892290099564}


In [68]:
df_aggvals = pd.read_csv(aggvals_csv).rename(columns={'Unnamed: 0': 'metric'}) # , 
df_aggvals
# df_aggvals.loc[df_aggvals[col_aggvals_year] == year]

,metric,Ag,Arterials & Suburban Corridors,Developing,Established Communities,Rural & Small Town Main Street,Rural Residential,Small-Town Established Communities,Urban core,REGION,year
0,WALKDESTSalljob,578.934547,5337.598083,945.143105,3208.991897,2121.642656,469.990251,1364.000857,64913.702561,3842.410878,2016
1,BIKEDESTSalljob,4181.014226,58753.268958,6511.420259,37113.506493,8340.183800,4359.477548,7127.598228,213458.712809,37354.023709,2016
2,AUTODESTSalljob,153980.014225,581272.181393,308544.671363,510067.786983,150264.423101,180455.785539,115470.053128,775596.880729,475047.245696,2016
3,TRANDESTSalljob,1998.225747,39399.817028,3391.266012,20656.833790,5562.764841,1608.656246,3942.737866,163632.287795,22213.193547,2016
4,WALKDESTSedu,1.083223,6.116137,1.678553,4.806067,3.959830,0.874987,2.835966,15.148187,4.638241,2016
5,BIKEDESTSedu,6.986049,53.544251,9.967731,40.360218,14.151564,7.208633,11.725413,116.927955,38.186083,2016
6,AUTODESTSedu,188.782305,598.965438,349.614989,526.154733,184.260412,222.133344,163.772323,822.954093,492.552901,2016
7,TRANDESTSedu,2.856191,29.992767,5.007700,19.988050,9.372950,2.745134,6.367323,67.712183,19.460658,2016
8,WALKDESTSpoi2,2.921800,23.408610,4.949238,17.393537,14.121013,2.691918,9.683589,84.084851,17.147198,2016
9,BIKEDESTSpoi2,24.681892,199.254586,35.425495,146.640604,49.186856,24.794275,43.011999,485.227235,140.100124,2016
